In [1]:
import gdal

import os.path as path
import stripy as stripy
import numpy as np

import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
gtiff = gdal.Open("/Users/lmoresi/+Codes/Python/quagmire/Notebooks/data/ETOPO1_Ice_c_geotiff.tif")

width = gtiff.RasterXSize
height = gtiff.RasterYSize
gt = gtiff.GetGeoTransform()
img = gtiff.GetRasterBand(1).ReadAsArray().T

img = np.fliplr(img)

imgLR = img[::10,::10]

In [3]:
# grid0 = stripy.icosahedral_mesh(include_face_points=False, refinement_levels=0)
# grid3 = stripy.icosahedral_mesh(include_face_points=False, refinement_levels=3)
# gridR = stripy.icosahedral_mesh(include_face_points=False, refinement_levels=8)

grid0 = stripy.triangulated_cube_mesh(refinement_levels=0)
grid3 = stripy.triangulated_cube_mesh(refinement_levels=3)
gridR = stripy.triangulated_cube_mesh(refinement_levels=8)

print grid0.npoints, grid3.npoints, gridR.npoints

8 386 393218


In [4]:
dlons = np.mod(np.degrees(gridR.lons), 360.0)
dlats = np.mod(np.degrees(gridR.lats)+90, 180.0)

ilons = img.shape[0] * dlons / 360.0
ilats = img.shape[1] * dlats / 180.0

icoords = np.stack((ilons, ilats))

from scipy import ndimage

meshheights = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

In [5]:
A = 6.0
B = 2.0

data = np.cos(A*gridR.lons) * np.cos(B*gridR.lats)

## Note the definition of the grad operator in spherical coords (lengths are shorter at high latitudes)

anddlons = (-A * np.sin(A*gridR.lons) * np.cos(B*gridR.lats)) 
anddlats =  -B * np.cos(A*gridR.lons) * np.sin(B*gridR.lats)

latcorr = np.cos(gridR.lats)
valid = latcorr != 0.0
latcorr[valid] = 1.0 / latcorr[valid]
anddlons *= latcorr

# anddlons[np.isnan(anddlons)] = 0.0

In [6]:
ddx, ddy, ddz = gridR.gradient(data, nit=10, tol=0.00001 )

AttributeError: 'triangulated_cube_mesh' object has no attribute 'gradient'

In [ ]:
ddlons, ddlats = gridR.transform_to_spherical(ddx, ddy, ddz)

ddlons2, ddlats2 = stripy.spherical.dxyz2dlonlat(gridR.x, gridR.y, gridR.z, ddx, ddy, ddz)



In [ ]:
ddlats2.max(), ddlons2.max(), anddlons.max()


In [ ]:
(anddlons-ddlons2).max()

In [ ]:
from LavaVu import lavavu

striangulationR = gridR
striangulation0 = grid3

wireframeI = striangulationR
trianglesI = striangulationR

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[2000,1000], near=-10.0)

# Core 

tris = lv.triangles("LAB",  wireframe=False, colour="#999999", opacity=1.0)
tris.vertices(striangulation0.points * 0.95 )
tris.indices(striangulation0.simplices)

# tris3 = lv.triangles("datagrid",  wireframe=False, colour="#77ff88", opacity=1.0)
# tris3.vertices(datagrid.points)
# tris3.indices(datagrid.simplices)
# tris3.values(data)
# tris3.colourmap(["#555599","#995555"])
                           
tris2 = lv.triangles("triangles",  wireframe=False, colour="#77ff88", opacity=0.5)
tris2.vertices(trianglesI.points )
tris2.indices(trianglesI.simplices)
tris2.values(anddlons-ddlons2)
#tris2.colourmap(["(-5.0)#555555", "(-0.001)#FFFFFF", "(0.0)#779977", "(0.1)#99AA99", "(1.0)#BBDDBB", "(5.0)#EEFFEE"] , logscale=False, range=[-7.0,5.0])   # Apply a built in colourmap
tris2.colourmap(['(-1.0)#FF0000','(-0.01)#FFDDDD','(0.01)#DDDDFF', '(1.0)#0000FF'], range=[-5.0,5.0])
tris2.colourbar()

# nodes = lv.points("DataPoints", pointsize=5.0, pointtype="shiny", colour="#448080", opacity=0.75)
# nodes.vertices(data_xyz*1.01)
# nodes.values(data)
# nodes.colourmap(["Blue","Red"])

lv.window()

tris.control.Checkbox(property='wireframe', label="Core - wireframe")
tris2.control.Checkbox(property='wireframe', label="Surface - wireframe")
# tris3.control.Checkbox(property='wireframe', label="Data - wireframe")


# tris2.control.show()

lv.control.Range('specular', range=(0,1), step=0.1, value=0)
lv.control.Checkbox(property='axis')
lv.control.ObjectList()
lv.control.show()

In [ ]:
## compute global slopes

ddx, ddy, ddz = gridR.gradient(meshheights/meshheights.max(), nit=10, tol=0.00001 )
dlo, dla = stripy.spherical.dxyz2dlonlat(gridR.x, gridR.y, gridR.z, ddx, ddy, ddz)

slope = np.sqrt(ddx**2+ddy**2+ddz**2)
slope2 = np.sqrt(dla**2+dlo**2)

In [ ]:

theta = np.arctan(slope)
theta2 = np.arctan(slope2)

In [ ]:
theta.mean()

In [ ]:
from LavaVu import lavavu

striangulationR = gridR
striangulation0 = grid3

wireframeI = striangulationR
trianglesI = striangulationR

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[2000,1000], near=-10.0)

# Core 

tris = lv.triangles("LAB",  wireframe=False, colour="#999999", opacity=1.0)
tris.vertices(striangulation0.points * 0.95 )
tris.indices(striangulation0.simplices)
                          
tris2 = lv.triangles("triangles",  wireframe=False, colour="#77ff88", opacity=0.5)
tris2.vertices(trianglesI.points )
tris2.indices(trianglesI.simplices)
tris2.values(theta)
#tris2.colourmap(["(-5.0)#555555", "(-0.001)#FFFFFF", "(0.0)#779977", "(0.1)#99AA99", "(1.0)#BBDDBB", "(5.0)#EEFFEE"] , logscale=False, range=[-7.0,5.0])   # Apply a built in colourmap
tris2.colourmap(['#FFFFEE','(1.2)#DDDDFF', '#0000FF'])
tris2.colourbar()

# nodes = lv.points("DataPoints", pointsize=5.0, pointtype="shiny", colour="#448080", opacity=0.75)
# nodes.vertices(data_xyz*1.01)
# nodes.values(data)
# nodes.colourmap(["Blue","Red"])

lv.window()

tris.control.Checkbox(property='wireframe', label="Core - wireframe")
tris2.control.Checkbox(property='wireframe', label="Surface - wireframe")
# tris3.control.Checkbox(property='wireframe', label="Data - wireframe")


# tris2.control.show()

lv.control.Range('specular', range=(0,1), step=0.1, value=0)
lv.control.Checkbox(property='axis')
lv.control.ObjectList()
lv.control.show()